<center>
    <img src="https://image.shutterstock.com/image-vector/regression-concept-keywords-people-icons-260nw-1820291330.jpg" width="600" alt="cognitiveclass.ai logo"  />
</center>

# Multiple Linear Regression

In [31]:
import pandas as pd 
import graphing
df = pd.read_excel('Data Collection\Petrol.xlsx', sheet_name='Combinedv2')

df1 = pd.read_excel('Data Collection\BFP.xlsx', sheet_name='Combined')
df1['Crude_Rands'] = (df1.Avg_Brent_Crude * df1.Ex_Rate)

#New dataset with new columns
to_drop = df[['BFP ', 'Fuel tax', 'Road accident fund']]
data = df.drop(to_drop, axis=1)
#Create a new column of Other cost
data['Other_cost']=data.iloc[:,2:13].sum(axis=1)
data_other = data['Other_cost']

# merge all into one dataset
all_df = [df1, to_drop, data_other]
dataset = pd.concat(all_df, axis=1)

#Drop duplicate BFP Column
dataset = dataset.drop(['BFP '], axis=1)
dataset.head()

,Date,BFP,Ex_Rate,Avg_Brent_Crude,Petrol,Crude_Rands,Fuel tax,Road accident fund,Other_cost
0,2010-01-01,406.263,7.5246,74.31,786.0,559.153026,150.0,64,161.737
1,2010-02-01,424.263,7.4735,76.84,804.0,574.263740,150.0,64,161.737
2,2010-03-01,430.563,7.6902,73.17,810.3,562.691934,150.0,64,161.737
3,2010-04-01,453.063,7.4753,78.89,858.3,589.726417,167.5,72,161.737
4,2010-05-01,465.063,7.3749,85.75,871.8,632.397675,167.5,72,163.237


#### Data Visualization

In [32]:
import plotly.express as px

fig = px.scatter(dataset, x ="Avg_Brent_Crude",y= "Petrol", title='Crude Oil vs BFP')
fig1 = px.scatter(dataset, x ="Ex_Rate", y= "Petrol", title='Exchange Rate vs BFP')

fig.show()
fig1.show()

In [33]:
import plotly.express as px
fig = px.scatter(dataset, x='Avg_Brent_Crude', y='BFP', trendline='ols', trendline_color_override='red', title='Crude Oil vs BFP')

fig1 = px.scatter(dataset, x='Ex_Rate', y='BFP', trendline='ols', trendline_color_override='red', title='Exchange Rate vs BFP')

fig.show()
fig1.show()


### Simple Linear Regression: Microsoft

In [34]:
import statsmodels.formula.api as smf
import graphing # custom graphing code from Microsft

for feature in ["Avg_Brent_Crude", "Ex_Rate"]:
    # Perform linear regression. This method takes care of
    # the entire fitting procedure for us.
    formula = "BFP ~ " + feature
    simple_model = smf.ols(formula = formula, data = dataset).fit()

    print(feature)
    print("R-squared:", simple_model.rsquared)
    
    # Show a graph of the result
    graphing.scatter_2D(dataset, label_x=feature, 
                                 label_y="BFP",
                                 title = feature,
                                 trendline=lambda x: simple_model.params[1] * x + simple_model.params[0],
                                 show=True)

Avg_Brent_Crude
R-squared: 0.18330218189942582


Ex_Rate
R-squared: 0.024549079692743447


### R-Squared

In [44]:
formula = "BFP ~ Avg_Brent_Crude"
BFP_trained_model = smf.ols(formula = formula, data = dataset).fit()
BFP_naive_model = smf.ols(formula = formula, data = dataset).fit()
BFP_naive_model.params[0] = dataset['BFP'].mean()
BFP_naive_model.params[1] = 0

print("naive R-squared:", BFP_naive_model.rsquared)
print("trained R-squared:", BFP_trained_model.rsquared)

# Show a graph of the result
graphing.scatter_2D(dataset, label_x="Avg_Brent_Crude", 
                                label_y="BFP",
                                title = "Naive model",
                                trendline=lambda x: dataset['BFP'].mean().repeat(len(x)), 
                                show=True)
# Show a graph of the result
graphing.scatter_2D(dataset, label_x="Avg_Brent_Crude", 
                                label_y="BFP",
                                title = "Trained model",
                                trendline=lambda x: BFP_trained_model.params[1] * x + BFP_trained_model.params[0])



naive R-squared: 0.0
trained R-squared: 0.18330218189942582


In [45]:
model = smf.ols(formula = "BFP ~ Avg_Brent_Crude + Ex_Rate", data = dataset).fit()

print("R-squared:", model.rsquared)

R-squared: 0.7873695057943867


In [37]:
import numpy as np
# Show a graph of the result
# this needs to be 3D, because we now have three variables in play: two features and one label

def predict( Avg_Brent_Crude , Ex_Rate):
    '''
    This converts given Avg_Brent_Crude and Ex_Change values into a prediction from the model
    '''
    # to make a prediction with statsmodels, we need to provide a dataframe
    # so create a dataframe with just the Avg_Brent_Crude and Ex_Rate variables
    df = pd.DataFrame(dict(Avg_Brent_Crude=[Avg_Brent_Crude], Ex_Rate=[Ex_Rate]))
    return model.predict(df)

# Create the surface graph
fig = graphing.surface(
    x_values=np.array([min(dataset.Avg_Brent_Crude), max(dataset.Avg_Brent_Crude)]),
    y_values=np.array([0, 25]),
    calc_z=predict,
    axis_title_x="Brent Crude Oil",
    axis_title_y="Exchange Rate",
    axis_title_z="BFP"
)

# Add our datapoints to it and display
fig.add_scatter3d(x=dataset.Avg_Brent_Crude, y=dataset.Ex_Rate, z=dataset.BFP, mode='markers')
fig.show()

In [38]:
# Print summary information
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BFP   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.784
Method:                 Least Squares   F-statistic:                     266.6
Date:                Mon, 14 Mar 2022   Prob (F-statistic):           3.88e-49
Time:                        11:02:33   Log-Likelihood:                -818.02
No. Observations:                 147   AIC:                             1642.
Df Residuals:                     144   BIC:                             1651.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        -558.2173     53.337    -10.466      0.000    -663.642    -452.793
Avg_Brent_Crude     7.2567      0.319     22.729      0.000       6.626       7.888
Ex_Rate            54.2112      2.680     20.226      0.000      48.913      59.509
==============================================================================
Omnibus:                       48.850   Durbin-Watson:                   0.318
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              139.339
Skew:                          -1.296   Prob(JB):                     5.53e-31
Kurtosis:                       7.003   Cond. No.                         822.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
BFP_trained_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    BFP   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     3.649
Date:                Mon, 14 Mar 2022   Prob (F-statistic):             0.0581
Time:                        11:02:33   Log-Likelihood:                -929.99
No. Observations:                 147   AIC:                             1864.
Df Residuals:                     145   BIC:                             1870.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    557.8531     44.450     12.550      0.000     469.999     645.707
Ex_Rate        6.8804      3.602      1.910      0.058      -0.238      13.999
==============================================================================
Omnibus:                        1.340   Durbin-Watson:                   0.175
Prob(Omnibus):                  0.512   Jarque-Bera (JB):                1.040
Skew:                           0.199   Prob(JB):                        0.595
Kurtosis:                       3.110   Cond. No.                         49.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Using Sklearn

Creat train and test dataset

In [40]:
cdf = dataset[['Avg_Brent_Crude', 'Ex_Rate', 'BFP']]

In [41]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

Mulltiple Regression Model

In [42]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
x = np.asanyarray(train[['Avg_Brent_Crude', 'Ex_Rate']])
y = np.asanyarray(train[['BFP']])
regr.fit (x, y)
# The coefficients
print ('Coefficients: ', regr.coef_)

Coefficients:  [[ 7.20872274 53.95743472]]


Prediction

In [43]:
y_hat= regr.predict(test[['Avg_Brent_Crude', 'Ex_Rate']])
x = np.asanyarray(test[['Avg_Brent_Crude', 'Ex_Rate']])
y = np.asanyarray(test[['BFP']])
print("RSS: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('R-Squared: %.2f' % regr.score(x, y))

RSS: 3218.57
R-Squared: 0.82


C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning:

X has feature names, but LinearRegression was fitted without feature names

